# Custom Gesture Creation

In [1]:
import os
import tensorflow as tf
from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

2024-03-25 23:10:03.096340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 23:10:03.096409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 23:10:03.098844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 23:10:03.109693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 23:10:04.993892: W tensorflow/compiler/tf2

## Get The Dataset

The dataset for gesture recognition in model maker requires the following format: `<dataset_path>/<label_name>/<img_name>.*`. In addition, one of the label names (label_names) must be none. The none label represents any gesture that isn't classified as one of the other gestures.

This example uses a rock paper scissors dataset sample which is downloaded from GCS.

In [2]:
dataset_path = "../data/Rock-Paper-Scissors-Data"

In [3]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

../data/Rock-Paper-Scissors-Data
['none', 'rock', 'paper', 'scissors']


In [4]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

/tmp/ipykernel_88582/544766458.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Load the dataset

Load the dataset located at dataset_path by using the Dataset.from_folder method. When loading the dataset, run the pre-packaged hand detection model from MediaPipe Hands to detect the hand landmarks from the images. Any images without detected hands are ommitted from the dataset. The resulting dataset will contain the extracted hand landmark positions from each image, rather than images themselves.

The HandDataPreprocessingParams class contains two configurable options for the data loading process:

`shuffle`: A boolean controlling whether to shuffle the dataset. Defaults to true.
`min_detection_confidence`: A float between 0 and 1 controlling the confidence threshold for hand detection.
`Split the dataset`: 80% for training, 10% for validation, and 10% for testing.

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams(min_detection_confidence=0.7)
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

## Train the model

Train the custom gesture recognizer by using the create method and passing in the training data, validation data, model options, and hyperparameters. For more information on model options and hyperparameters, see the Hyperparameters section below.

In [7]:
hparams = gesture_recognizer.HParams(learning_rate=0.003, epochs=40, export_dir="../tasks")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 4)                 516       
 out (Dense)                                                     
                                                             

INFO:tensorflow:Training the models...


Epoch 1/40
189/189 [==============================] - 3s 9ms/step - loss: 0.7857 - categorical_accuracy: 0.4286 - val_loss: 0.2779 - val_categorical_accuracy: 0.7872 - lr: 0.0030
Epoch 2/40
189/189 [==============================] - 1s 6ms/step - loss: 0.5652 - categorical_accuracy: 0.5661 - val_loss: 0.2663 - val_categorical_accuracy: 0.8511 - lr: 0.0030
Epoch 3/40
189/189 [==============================] - 1s 6ms/step - loss: 0.5005 - categorical_accuracy: 0.6323 - val_loss: 0.2728 - val_categorical_accuracy: 0.8511 - lr: 0.0029
Epoch 4/40
189/189 [==============================] - 1s 7ms/step - loss: 0.4766 - categorical_accuracy: 0.6481 - val_loss: 0.2541 - val_categorical_accuracy: 0.8298 - lr: 0.0029
Epoch 5/40
189/189 [==============================] - 1s 6ms/step - loss: 0.4547 - categorical_accuracy: 0.6640 - val_loss: 0.2700 - val_categorical_accuracy: 0.8298 - lr: 0.0029
Epoch 6/40
189/189 [==============================] - 1s 6ms/step - loss: 0.4377 - categorical_accuracy: 

## Evaluate the model performance

After training the model, evaluate it on a test dataset and print the loss and accuracy metrics.


In [8]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

48/48 [==============================] - 1s 4ms/step - loss: 0.2055 - categorical_accuracy: 0.8750
Test loss:0.20546700060367584, Test accuracy:0.875


## Export to Tensorflow Lite Model

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label file.


In [9]:
model.export_model("../tasks/gesture_recognizer.task")

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite
INFO:tensorflow:Assets written to: /tmp/tmpxt28goh3/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpxt28goh3/saved_model/assets
2024-03-25 23:20:14.852953: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-25 23:20:14.853034: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-25 23:20:14.853867: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxt28goh3/saved_model
2024-03-25 23:20:14.855780: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-25 23:20:14.855814: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpxt28goh3/saved_model
2024-03-25 23:20:14.859106: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-03-25 23:20:14.860695: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-25 23:20:14.899826: I tensorflow/cc/saved_model/

## Hyperparameters

You can further customize the model using the GestureRecognizerOptions class, which has two optional parameters for ModelOptions and HParams. Use the ModelOptions class to customize parameters related to the model itself, and the HParams class to customize other parameters related to training and saving the model.

ModelOptions has one customizable parameter that affects accuracy:

`dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.
`layer_widths`: A list of hidden layer widths for the gesture model. Each element in the list will create a new hidden layer with the specified width. The hidden layers are separated with BatchNorm, Dropout, and ReLU. Defaults to an empty list(no hidden layers).

HParams has the following list of customizable parameters which affect model accuracy:

`learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
`batch_size`: Batch size for training. Defaults to 2.
`epochs`: Number of training iterations over the dataset. Defaults to 10.
`steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
`shuffle`: True if the dataset is shuffled before training. Defaults to False.
`lr_decay`: Learning rate decay to use for gradient descent training. Defaults to 0.99.
`gamma`: Gamma parameter for focal loss. Defaults to 2

Additional HParams parameter that does not affect model accuracy:

`export_dir`: The location of the model checkpoint files and exported model files.

For example, the following trains a new model with the dropout_rate of 0.2 and learning rate of 0.003.

In [ ]:
hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="exported_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

In [ ]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")